In [ ]:
# The passive floating kelp individual will be treated as an Argo Float in this approach
# The script for Agro floats has behaviour of sinking, and sinking rates which can be adpated later
# but also samples environmental data that can be used as input for the "growth/ecklonia" model

In [ ]:
# First load in the needed modules which come standard with the MiniConda3 download

In [ ]:
%matplotlib inline
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, plotTrajectoriesFile, ErrorCode, Variable
from datetime import timedelta, datetime
import numpy as np

In [ ]:
# Then instatiate a FieldSet with the velocity field data from GlobCurrent dataset.

In [ ]:
filenames = {'U': "GlobCurrent_example_data/20*.nc", # Define the names of the files containing zonal (U) and meridional (V) velocities
             'V': "GlobCurrent_example_data/20*.nc"} # The filenames can be the same for U and V
variables = {'U': 'eastward_eulerian_current_velocity',
             'V': 'northward_eulerian_current_velocity'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)
#fieldset.mindepth = fieldset.U.depth[0]  # uppermost layer in the hydrodynamic data

In [ ]:
class KelpParticle(JITParticle):
    drag = Variable('drag', dtype=np.int32, initial=1)

def KelpVel(particle, fieldset, time):
    value = fieldset.U[time, particle.lon, particle.lat, particle.depth]
    particle.lon += U*drag*particle.dt

In [ ]:
# Initiate a ParticleSet composed of JITParticles

In [ ]:
# Execute the simulation

In [ ]:
# Initiate one Argo float in the Agulhas Current
pset = ParticleSet(fieldset=fieldset, pclass=KelpParticle, lon=[18], lat=[-34], depth=[0],time=datetime(2002, 1, 1))

# combine Argo vertical movement kernel with built-in Advection kernel
kernels = pset.Kernel(AdvectionRK4) + (KelpVel)

# Create a ParticleFile object to store the output
output_file = pset.ParticleFile(name="custom_float", outputdt=timedelta(minutes=30))

# Now execute the kernels for 30 days, saving data every 30 minutes
pset.execute(kernels, runtime=timedelta(days=7), dt=timedelta(minutes=30), output_file=output_file)

In [ ]:
# Visualise simulation using plotParticles script. Particles can be plotted on top of velocity fields 
# using the tracerfile, tracerfield, ect keywords

In [ ]:
output_file.export()
plotTrajectoriesFile('custom_float.nc',
                     tracerfile='parcels_examples/GlobCurrent_example_data/20020101000000-GLOBCURRENT-L4-CUReul_hs-ALT_SUM-v02.0-fv01.0.nc',
                     tracerlon='lon',
                     tracerlat='lat',
                     tracerfield='eastward_eulerian_current_velocity')

In [ ]:
# Obtaining data from float

In [ ]:
%matplotlib inline
import netCDF4
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
output_file.export()  # export the trajectory data to a netcdf file

nc = netCDF4.Dataset("custom_float.nc")
x = nc.variables["lon"][:].squeeze()
y = nc.variables["lat"][:].squeeze()
z = nc.variables["z"][:].squeeze()
nc.close()

# Visualise output

fig = plt.figure(figsize=(13,10))
ax = plt.axes(projection='3d')
cb = ax.scatter(x, y, z, c=z, s=20, marker="o")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_zlabel("Depth (m)")
ax.set_zlim(np.max(z),0)
plt.show()


In [ ]:
# Save the output file
# Trajectory of particle set for 10 days using 4th order Runge-Kutta scheme
# Trajectories are plotted over Eastward eulerian current velocity

In [ ]:
pset.show(field = 'vector', savefile='kelp_float')

In [ ]:
# Visualise trajectory by means of animation

In [ ]:
plotTrajectoriesFile('kelp_float.nc', mode='movie2d_notebook')